In [1]:
#importing modules
import cv2
import numpy as np
import pandas as pd
import os
from skimage.feature import local_binary_pattern
from scipy.stats import itemfreq
from sklearn.preprocessing import normalize
import csv
import pickle
import joblib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import sklearn.metrics as metrics

In [5]:
pip install xgboost

     |████████████████████████████████| 297.1 MB 2.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
#lbp definiton for positve trainging images
def lbp_positive(folder_name,radius, no_points ):
    i=0
    label=1
    for filename in os.listdir(folder_name):
        #path
        path=os.path.join(folder_name,filename)
        
        #read the image
        a=cv2.imread(path)
        
        #change the image from rgb to grayscale
        a = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
        
        #apply gaussian filter
        img_gaussian = cv2.GaussianBlur(a,(3,3),0)
        
        #define the kernel for prewitt edge detection operator
        kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
        kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
        img_prewittx = cv2.filter2D(img_gaussian, -1, kernelx)
        img_prewitty = cv2.filter2D(img_gaussian, -1, kernely)
        prewitt = img_prewittx + img_prewitty
        
        #apply canny edge detection.
        img_canny = cv2.Canny(a,100,200)
        final_img = prewitt + img_canny
        
        #set radius for LBP
        #radius = 3
        
        # Number of points to be considered as neighbourers 
        #no_points = 8 * radius
        
        # Uniform LBP is used
        lbp = local_binary_pattern(final_img, no_points, radius, method='uniform')
        
        # Calculate the histogram
        x = itemfreq(lbp.ravel())
        
        # Normalize the histogram
        hist = x[:, 1]/sum(x[:, 1])
        
        hist.reshape(26,-1)
        hist = np.append(hist, label)
        #writing features to csv files
        with open(r'finalcsv.csv', 'a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(hist)
        i=i+1
        if i==3000:
            break;

In [3]:
#lbp definition for negative training images
def lbp_negative(folder_name,radius, no_points ):
    i=0
    label=0
    for filename in os.listdir(folder_name):
        #path
        path=os.path.join(folder_name,filename)
        
        #read the image
        a=cv2.imread(path)
        
        #change the image from rgb to grayscale
        a = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
        
        #apply gaussian filter
        img_gaussian = cv2.GaussianBlur(a,(3,3),0)
        
        #define the kernel for prewitt edge detection operator
        kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
        kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])
        img_prewittx = cv2.filter2D(img_gaussian, -1, kernelx)
        img_prewitty = cv2.filter2D(img_gaussian, -1, kernely)
        prewitt = img_prewittx + img_prewitty
        
        #apply canny edge detection.
        img_canny = cv2.Canny(a,100,200)
        final_img = prewitt + img_canny
        
        #set radius for LBP
        #radius = 3
        
        # Number of points to be considered as neighbourers 
        #no_points = 8 * radius
        
        # Uniform LBP is used
        lbp = local_binary_pattern(final_img, no_points, radius, method='uniform')
        
        # Calculate the histogram
        x = itemfreq(lbp.ravel())
        
        # Normalize the histogram
        hist = x[:, 1]/sum(x[:, 1])
        
        hist.reshape(26,-1)
        hist = np.append(hist, label)
        #writing features to csv files
        with open(r'finalcsv.csv', 'a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(hist)
        i=i+1
        if i==3000:
            break;

In [ ]:
#calling the lbp function for positive trainging images
lbp_positive(r"/home/admin-/Downloads/cr",3,24)

/tmp/ipykernel_49692/8429825.py:39: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  x = itemfreq(lbp.ravel())


In [8]:
#calling the lbp function for negative training images
lbp_negative(r"/home/admin-/Downloads/nc",3,24)

/tmp/ipykernel_15888/2789158279.py:39: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  x = itemfreq(lbp.ravel())


In [9]:
#reading the csv file into 'data' variable
data= pd.read_csv('/home/admin-/EDAI project/finalcsv.csv',header=None)
data

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,0.053108,0.040787,0.023000,0.014037,0.008537,0.006304,0.005121,0.006075,0.005525,0.006383,...,0.004738,0.004075,0.004892,0.006092,0.007958,0.011371,0.011550,0.276442,0.431354,1.0
1,0.066246,0.036338,0.026908,0.018775,0.011096,0.007471,0.006437,0.006104,0.005587,0.006308,...,0.005208,0.004733,0.005692,0.006917,0.008833,0.012483,0.014238,0.244654,0.450087,1.0
2,0.061510,0.035275,0.031329,0.019758,0.010398,0.006463,0.004985,0.005521,0.005360,0.006329,...,0.003854,0.003400,0.004283,0.005031,0.007473,0.009273,0.009171,0.304250,0.382694,1.0
3,0.053752,0.038779,0.023873,0.016562,0.010079,0.007347,0.005662,0.005728,0.005366,0.006193,...,0.004749,0.004471,0.005027,0.005795,0.007293,0.009873,0.011619,0.308381,0.408731,1.0
4,0.078686,0.032784,0.021990,0.015850,0.009175,0.005313,0.003865,0.003338,0.002805,0.002540,...,0.002673,0.002904,0.004064,0.005693,0.008138,0.013242,0.021692,0.216302,0.532604,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,0.060769,0.026655,0.029755,0.022017,0.012529,0.007998,0.005773,0.005405,0.005297,0.005645,...,0.003056,0.003116,0.003679,0.004906,0.005641,0.006975,0.006560,0.408784,0.334823,0.0
5396,0.051875,0.022832,0.028362,0.020497,0.013845,0.007870,0.005258,0.005391,0.005351,0.006600,...,0.009676,0.004586,0.005591,0.006848,0.008971,0.009760,0.006993,0.334319,0.382041,0.0
5397,0.070167,0.035263,0.024682,0.019187,0.012544,0.009085,0.007278,0.007050,0.007026,0.007122,...,0.003926,0.003842,0.004704,0.006392,0.010294,0.013598,0.017284,0.226108,0.459387,0.0
5398,0.058628,0.026606,0.028126,0.024031,0.016965,0.011444,0.009329,0.008857,0.008902,0.009788,...,0.006022,0.005862,0.007096,0.008115,0.009976,0.012759,0.010901,0.279314,0.381289,0.0


In [10]:
#splitting the data into x and y with 26 features for x and label column for y.
x = data.iloc[:,0:26].values
y = data.iloc[:,26].values

In [11]:
#scaling the data with the help of standardization
from sklearn.preprocessing import StandardScaler
std_obj = StandardScaler()
scalardata = std_obj.fit_transform(x)

In [12]:
#viewing the standardized data
scalardata

array([[-0.36476046,  1.52397565, -0.80116289, ...,  0.0164222 ,
        -0.59825639,  0.59356418],
       [ 0.90452673,  0.84175173,  0.20363608, ...,  0.6501561 ,
        -0.95707242,  0.86525568],
       [ 0.4470117 ,  0.67886118,  1.3401944 , ..., -0.54460425,
        -0.28435703, -0.11216294],
       ...,
       [ 1.28333569,  0.67699933, -0.3688367 , ...,  1.36861705,
        -1.16641631,  1.00012252],
       [ 0.16854777, -0.65017321,  0.51670261, ..., -0.13665465,
        -0.56583229, -0.1325424 ],
       [ 1.35014969,  0.4412341 ,  0.00398991, ...,  1.11758012,
        -1.10034034,  0.77475948]])

In [13]:
#importing sklearn function for pca
from sklearn.decomposition import PCA

In [14]:
#applying PCA to reduce dimensions
pca = PCA(n_components=None)
pca.fit(scalardata)

PCA()

In [15]:
new_scalardata = pca.transform(scalardata)
new_scalardata

array([[ 1.28476084e-01, -9.53553025e-01,  2.48426100e+00, ...,
         6.52763794e-02, -1.06223038e-03,  2.56756419e-15],
       [ 7.30490027e-01, -2.33696554e+00, -1.97356780e-01, ...,
         3.94571350e-02,  4.15240142e-03,  2.31948814e-15],
       [ 5.63921676e-01,  7.24388245e-01,  9.08377478e-01, ...,
        -2.59474032e-02, -1.33779597e-02,  2.57457776e-15],
       ...,
       [ 1.06160004e+00, -2.83266057e+00, -6.17287567e-01, ...,
        -3.58438991e-02, -1.77834303e-01,  5.77652571e-16],
       [ 6.32558139e+00, -8.44123900e-01, -1.19717324e+00, ...,
         1.28985857e-01, -2.40708418e-02,  1.84443629e-15],
       [ 1.83370913e+00, -3.29135138e+00, -1.93554363e-01, ...,
        -8.46339129e-02,  3.61822953e-02,  1.95997319e-15]])

In [16]:
new_scalardata.shape

(5400, 26)

In [17]:
new_scalardata = pd.DataFrame(new_scalardata)

In [18]:
new_scalardata

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.128476,-0.953553,2.484261,-0.938782,0.429984,0.378784,-0.554473,-0.254278,0.013919,0.327864,...,-0.080958,-0.214992,0.242315,0.214886,-0.084963,0.029901,0.056115,0.065276,-0.001062,2.567564e-15
1,0.730490,-2.336966,-0.197357,-0.099710,0.042322,0.420716,-0.017422,0.020909,-0.061947,0.226997,...,-0.073002,-0.184407,0.052280,-0.046525,0.160226,-0.185574,0.031686,0.039457,0.004152,2.319488e-15
2,0.563922,0.724388,0.908377,-2.165524,2.412024,-0.565519,0.513390,-0.153776,-0.250971,0.204393,...,-0.034748,0.018079,0.238624,-0.152767,-0.066965,0.014533,0.053750,-0.025947,-0.013378,2.574578e-15
3,-0.057953,-0.408774,1.099041,-0.598434,-0.205021,0.854135,-0.445132,-0.308015,0.131580,-0.219747,...,-0.035859,-0.062878,0.006159,-0.005593,0.018508,-0.058088,-0.012220,-0.013122,-0.073147,-3.356616e-15
4,-5.676309,-4.075108,-1.300399,0.403183,-0.811104,-0.677178,0.810935,-0.066891,0.501818,0.045705,...,0.088755,-0.010662,-0.012330,-0.003450,0.027476,-0.008440,-0.006929,-0.067044,0.012137,9.009987e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,-1.677530,2.298283,-2.470454,-0.711787,-0.034619,0.018352,0.011348,-0.178542,-0.217511,-0.075501,...,-0.093042,-0.140242,-0.123634,0.072497,0.001323,-0.166589,0.024089,-0.090090,-0.082518,-8.750743e-16
5396,2.563273,0.642886,-0.114482,2.374667,0.664604,2.437336,2.402342,0.822720,0.693345,0.205482,...,0.276381,0.084063,-0.088606,-0.051174,-0.007274,-0.134906,-0.096434,0.157186,-0.123722,1.570965e-15
5397,1.061600,-2.832661,-0.617288,-0.625660,-1.056084,-1.082804,-0.204459,0.854091,-0.230621,0.008330,...,0.214487,0.256306,-0.181507,-0.206344,0.042204,-0.142217,-0.230129,-0.035844,-0.177834,5.776526e-16
5398,6.325581,-0.844124,-1.197173,0.377364,-1.109529,-0.497345,-0.451555,0.381003,-0.173963,-0.055648,...,-0.288996,-0.059291,0.138696,0.038389,0.143562,-0.148784,-0.027999,0.128986,-0.024071,1.844436e-15


In [19]:
new_scalardata.shape

(5400, 26)

In [20]:
#Initially the explained variance sum of the full data with 26 features comes out to be 1.
print(pca.explained_variance_ratio_.sum())

1.0000000000000002


In [22]:
#Initially the explained variance sum of the full data with 26 features comes out to be 1.
print(pca.explained_variance_ratio_)

[5.79904084e-01 2.12951366e-01 9.22497627e-02 3.85436929e-02
 3.05158381e-02 1.26835505e-02 7.44364055e-03 5.44939073e-03
 3.63468650e-03 3.12004640e-03 2.77181392e-03 2.03501420e-03
 1.18107603e-03 1.10759523e-03 9.64386871e-04 8.11927700e-04
 6.64380796e-04 6.33289567e-04 6.07999103e-04 5.95410751e-04
 5.29716948e-04 4.73836118e-04 3.99085900e-04 3.82870260e-04
 3.45538716e-04 1.66193439e-31]


In [23]:
#choosing only 18 columns from the data.
pca = PCA(n_components=20)
pca.fit(scalardata)

PCA(n_components=20)

In [25]:
# save the model to disk
filename = 'PCA_3_Model.sav'
pickle.dump(pca, open(filename, 'wb'))

In [26]:
new_scalardata = pca.transform(scalardata)
new_scalardata

array([[ 0.12847608, -0.95355303,  2.484261  , ..., -0.21499233,
         0.24231463,  0.21488625],
       [ 0.73049003, -2.33696554, -0.19735678, ..., -0.18440678,
         0.05228022, -0.04652485],
       [ 0.56392168,  0.72438825,  0.90837748, ...,  0.0180794 ,
         0.23862358, -0.15276742],
       ...,
       [ 1.06160004, -2.83266057, -0.61728757, ...,  0.25630618,
        -0.18150717, -0.20634408],
       [ 6.32558139, -0.8441239 , -1.19717324, ..., -0.05929087,
         0.13869582,  0.03838929],
       [ 1.83370913, -3.29135138, -0.19355436, ...,  0.19035461,
        -0.03673238,  0.06288735]])

In [27]:
new_scalardata.shape

(5400, 20)

In [28]:
#explained variance sum of the data with 18 columns comes out to be 99%.
print(pca.explained_variance_ratio_.sum()) 

0.9978689520592077


In [29]:
#converting the data into dataframe
new_scalardata = pd.DataFrame(new_scalardata)
new_scalardata

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.128476,-0.953553,2.484261,-0.938782,0.429984,0.378784,-0.554473,-0.254278,0.013919,0.327864,-0.278600,0.152302,-0.079568,-0.250151,-0.107531,-0.245801,-0.080958,-0.214992,0.242315,0.214886
1,0.730490,-2.336966,-0.197357,-0.099710,0.042322,0.420716,-0.017422,0.020909,-0.061947,0.226997,0.081446,-0.255886,0.141114,-0.059088,0.066955,0.151233,-0.073002,-0.184407,0.052280,-0.046525
2,0.563922,0.724388,0.908377,-2.165524,2.412024,-0.565519,0.513390,-0.153776,-0.250971,0.204393,-0.065595,-0.168926,-0.169644,-0.052725,-0.057067,-0.293405,-0.034748,0.018079,0.238624,-0.152767
3,-0.057953,-0.408774,1.099041,-0.598434,-0.205021,0.854135,-0.445132,-0.308015,0.131580,-0.219747,-0.024985,-0.049617,0.183280,-0.065502,-0.033485,0.071344,-0.035859,-0.062878,0.006159,-0.005593
4,-5.676309,-4.075108,-1.300399,0.403183,-0.811104,-0.677178,0.810935,-0.066891,0.501818,0.045705,0.508965,0.027678,-0.092043,0.095098,-0.057876,0.002760,0.088755,-0.010662,-0.012330,-0.003450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,-1.677530,2.298283,-2.470454,-0.711787,-0.034619,0.018352,0.011348,-0.178542,-0.217511,-0.075501,-0.060768,-0.098593,-0.068376,0.049699,0.072006,-0.001584,-0.093042,-0.140242,-0.123634,0.072497
5396,2.563273,0.642886,-0.114482,2.374667,0.664604,2.437336,2.402342,0.822720,0.693345,0.205482,-1.071011,-0.191605,-0.349451,0.191343,0.571275,-0.203902,0.276381,0.084063,-0.088606,-0.051174
5397,1.061600,-2.832661,-0.617288,-0.625660,-1.056084,-1.082804,-0.204459,0.854091,-0.230621,0.008330,-0.750066,-0.007717,0.205836,-0.230538,0.077988,-0.344476,0.214487,0.256306,-0.181507,-0.206344
5398,6.325581,-0.844124,-1.197173,0.377364,-1.109529,-0.497345,-0.451555,0.381003,-0.173963,-0.055648,-0.103346,0.031517,0.116338,-0.223165,-0.094717,0.048993,-0.288996,-0.059291,0.138696,0.038389


In [30]:
#appending the label column to the dataframe.
label_column = pd.concat([new_scalardata, pd.DataFrame(y)],axis=1)
label_column

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,0
0,0.128476,-0.953553,2.484261,-0.938782,0.429984,0.378784,-0.554473,-0.254278,0.013919,0.327864,...,0.152302,-0.079568,-0.250151,-0.107531,-0.245801,-0.080958,-0.214992,0.242315,0.214886,1.0
1,0.730490,-2.336966,-0.197357,-0.099710,0.042322,0.420716,-0.017422,0.020909,-0.061947,0.226997,...,-0.255886,0.141114,-0.059088,0.066955,0.151233,-0.073002,-0.184407,0.052280,-0.046525,1.0
2,0.563922,0.724388,0.908377,-2.165524,2.412024,-0.565519,0.513390,-0.153776,-0.250971,0.204393,...,-0.168926,-0.169644,-0.052725,-0.057067,-0.293405,-0.034748,0.018079,0.238624,-0.152767,1.0
3,-0.057953,-0.408774,1.099041,-0.598434,-0.205021,0.854135,-0.445132,-0.308015,0.131580,-0.219747,...,-0.049617,0.183280,-0.065502,-0.033485,0.071344,-0.035859,-0.062878,0.006159,-0.005593,1.0
4,-5.676309,-4.075108,-1.300399,0.403183,-0.811104,-0.677178,0.810935,-0.066891,0.501818,0.045705,...,0.027678,-0.092043,0.095098,-0.057876,0.002760,0.088755,-0.010662,-0.012330,-0.003450,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,-1.677530,2.298283,-2.470454,-0.711787,-0.034619,0.018352,0.011348,-0.178542,-0.217511,-0.075501,...,-0.098593,-0.068376,0.049699,0.072006,-0.001584,-0.093042,-0.140242,-0.123634,0.072497,0.0
5396,2.563273,0.642886,-0.114482,2.374667,0.664604,2.437336,2.402342,0.822720,0.693345,0.205482,...,-0.191605,-0.349451,0.191343,0.571275,-0.203902,0.276381,0.084063,-0.088606,-0.051174,0.0
5397,1.061600,-2.832661,-0.617288,-0.625660,-1.056084,-1.082804,-0.204459,0.854091,-0.230621,0.008330,...,-0.007717,0.205836,-0.230538,0.077988,-0.344476,0.214487,0.256306,-0.181507,-0.206344,0.0
5398,6.325581,-0.844124,-1.197173,0.377364,-1.109529,-0.497345,-0.451555,0.381003,-0.173963,-0.055648,...,0.031517,0.116338,-0.223165,-0.094717,0.048993,-0.288996,-0.059291,0.138696,0.038389,0.0


In [31]:
#saving the dataframe as as csv file.
csv_data=label_column.to_csv('Downloads\FinalPCA(p&c).csv', mode='a',header=False,index=False)

In [32]:
#reading the final training data.
train_data = pd.read_csv('Downloads\FinalPCA(p&c).csv',header=None)

In [33]:
train_data

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.128476,-0.953553,2.484261,-0.938782,0.429984,0.378784,-0.554473,-0.254278,0.013919,0.327864,...,0.152302,-0.079568,-0.250151,-0.107531,-0.245801,-0.080958,-0.214992,0.242315,0.214886,1.0
1,0.730490,-2.336966,-0.197357,-0.099710,0.042322,0.420716,-0.017422,0.020909,-0.061947,0.226997,...,-0.255886,0.141114,-0.059088,0.066955,0.151233,-0.073002,-0.184407,0.052280,-0.046525,1.0
2,0.563922,0.724388,0.908377,-2.165524,2.412024,-0.565519,0.513390,-0.153776,-0.250971,0.204393,...,-0.168926,-0.169644,-0.052725,-0.057067,-0.293405,-0.034748,0.018079,0.238624,-0.152767,1.0
3,-0.057953,-0.408774,1.099041,-0.598434,-0.205021,0.854135,-0.445132,-0.308015,0.131580,-0.219747,...,-0.049617,0.183280,-0.065502,-0.033485,0.071344,-0.035859,-0.062878,0.006159,-0.005593,1.0
4,-5.676309,-4.075108,-1.300399,0.403183,-0.811104,-0.677178,0.810935,-0.066891,0.501818,0.045705,...,0.027678,-0.092043,0.095098,-0.057876,0.002760,0.088755,-0.010662,-0.012330,-0.003450,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,-1.677530,2.298283,-2.470454,-0.711787,-0.034619,0.018352,0.011348,-0.178542,-0.217511,-0.075501,...,-0.098593,-0.068376,0.049699,0.072006,-0.001584,-0.093042,-0.140242,-0.123634,0.072497,0.0
5396,2.563273,0.642886,-0.114482,2.374667,0.664604,2.437336,2.402342,0.822720,0.693345,0.205482,...,-0.191605,-0.349451,0.191343,0.571275,-0.203902,0.276381,0.084063,-0.088606,-0.051174,0.0
5397,1.061600,-2.832661,-0.617288,-0.625660,-1.056084,-1.082804,-0.204459,0.854091,-0.230621,0.008330,...,-0.007717,0.205836,-0.230538,0.077988,-0.344476,0.214487,0.256306,-0.181507,-0.206344,0.0
5398,6.325581,-0.844124,-1.197173,0.377364,-1.109529,-0.497345,-0.451555,0.381003,-0.173963,-0.055648,...,0.031517,0.116338,-0.223165,-0.094717,0.048993,-0.288996,-0.059291,0.138696,0.038389,0.0


In [34]:
#assigning x the columns from 1 to 18 for training
x = train_data.iloc[:,0:20].values
print("X values")
print(x)

#assigning y with the column "Class" as target variable
y = train_data.iloc[:,20]
print("Y values")
print(y)

X values
[[ 0.12847608 -0.95355303  2.484261   ... -0.21499233  0.24231463
   0.21488625]
 [ 0.73049003 -2.33696554 -0.19735678 ... -0.18440678  0.05228022
  -0.04652485]
 [ 0.56392168  0.72438825  0.90837748 ...  0.0180794   0.23862358
  -0.15276742]
 ...
 [ 1.06160004 -2.83266057 -0.61728757 ...  0.25630618 -0.18150717
  -0.20634408]
 [ 6.32558139 -0.8441239  -1.19717324 ... -0.05929087  0.13869582
   0.03838929]
 [ 1.83370913 -3.29135138 -0.19355436 ...  0.19035461 -0.03673238
   0.06288735]]
Y values
0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
5395    0.0
5396    0.0
5397    0.0
5398    0.0
5399    0.0
Name: 20, Length: 5400, dtype: float64


In [35]:
#Dataset split into train and test with 80% Training and 20% Testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.20, random_state=0)

In [46]:
#Decision Tree Classifier
from sklearn.metrics import accuracy_score
#Assign model with Decision Tree classifier
model_dt = DecisionTreeClassifier(max_depth=11)
#training the model with the Training Variables 
model_dt.fit(x_train, y_train)
#dumping the decision tree classifier model to the disk
joblib.dump(model_dt,"model_dt")
#predicting the target variable using testing variables
y_pred1 = model_dt.predict(x_test)
#Results
print("Decision Tree Results")
print("Decision Tree Accuracy: ",accuracy_score(y_test, y_pred1)*100,"%")
print("Train Accuracy:",model_dt.score(x_train, y_train))
print("Test Accuracy:",model_dt.score(x_test, y_test))
print("Precision Score: ",metrics.precision_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred1, pos_label='positive', average='micro')) # true positive rate, Sensitivity
print("F2 Score: ",metrics.fbeta_score(y_test, y_pred1, pos_label='positive', average='micro', beta=2.0))
print("F1 Score: ",metrics.f1_score(y_test, y_pred1, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred1))

Decision Tree Results
Decision Tree Accuracy:  76.38888888888889 %
Train Accuracy: 0.9569444444444445
Test Accuracy: 0.7638888888888888
Precision Score:  0.7638888888888888
Recall Score:  0.7638888888888888
F2 Score:  0.7638888888888888
F1 Score:  0.763888888888889
Confusion Matrix: 
[[411 130]
 [125 414]]


/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admi

In [49]:
#Random Forest classifier
model_rf = RandomForestClassifier(n_estimators = 100)
#training the model with train variables
model_rf.fit(x_train, y_train)
#dumping the random forest classifier model to the disk
joblib.dump(model_rf,"model_rf")
#predicting with the trained random forest model
y_pred2 = model_rf.predict(x_test)
print("Random Forest Clasifier")
print("Train Accuracy:",model_rf.score(x_train, y_train))
print("Test Accuracy:",model_rf.score(x_test, y_test)*100)
print("Precision Score: ",metrics.precision_score(y_test, y_pred2, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred2, pos_label='positive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred2, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred2,))

Random Forest Clasifier
Train Accuracy: 0.999537037037037
Test Accuracy: 81.48148148148148
Precision Score:  0.8148148148148148
Recall Score:  0.8148148148148148
F1 Score:  0.8148148148148148
Confusion Matrix: 
[[450  91]
 [109 430]]


/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [62]:
#KNN classifier
model_knn = KNeighborsClassifier(n_neighbors = 8)
model_knn.fit(x_train, y_train)
#dumping the KNN classifier model to the disk.
joblib.dump(model_knn,"model_knn")
y_pred3 = model_knn.predict(x_test)
print("KNN")
print("Train Accuracy:",model_knn.score(x_train, y_train))
print("Test Accuracy:",model_knn.score(x_test, y_test)*100)
print("Precision Score: ",metrics.precision_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred3, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred3,))

KNN
Train Accuracy: 0.819212962962963
Test Accuracy: 76.11111111111111
Precision Score:  0.7611111111111111
Recall Score:  0.7611111111111111
F1 Score:  0.7611111111111111
Confusion Matrix: 
[[448  93]
 [165 374]]


/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [65]:
#svm classifier
model_svm = svm.SVC(kernel='rbf') # rbf Kernel
model_svm.fit(x_train, y_train)
#dumping the svm classifier model to the disk
joblib.dump(model_svm,"model_svm")
#prediction
y_pred4 = model_svm.predict(x_test)
print("SVM Linear kernel")
print("Train Accuracy:",model_svm.score(x_train, y_train))
print("Test Accuracy:",model_svm.score(x_test, y_test)*100)
print("Precision Score: ",metrics.precision_score(y_test, y_pred4, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred4, pos_label='positive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred4, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred4,))

SVM Linear kernel
Train Accuracy: 0.8319444444444445
Test Accuracy: 80.0
Precision Score:  0.8
Recall Score:  0.8
F1 Score:  0.8000000000000002
Confusion Matrix: 
[[446  95]
 [121 418]]


/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [67]:
#gaussian naive bayes classifier
model_nb = GaussianNB().fit(x_train, y_train)
#dumping the naive bayes classifier model to the disk
joblib.dump(model_nb,"model_nb")
y_pred8 = model_nb.predict(x_test)
print("Gaussian NB")
print("Train Accuracy:",model_nb.score(x_train, y_train))
print("Test Accuracy:",model_nb.score(x_test, y_test)*100)
print("Precision Score: ",metrics.precision_score(y_test, y_pred8, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred8, pos_label='positive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred8, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred8,))

Gaussian NB
Train Accuracy: 0.7951388888888888
Test Accuracy: 76.85185185185185
Precision Score:  0.7685185185185185
Recall Score:  0.7685185185185185
F1 Score:  0.7685185185185186
Confusion Matrix: 
[[434 107]
 [143 396]]


/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "


In [69]:
#xgboost classifier
model_xgboost = XGBClassifier().fit(x_train, y_train)
#dumping the xgboost classifier model to the disk.
joblib.dump(model_nb,"model_xgboost")
#prediction
y_pred5 = model_xgboost.predict(x_test)
print("XGboost")
print("Train Accuracy:",model_xgboost.score(x_train, y_train))
print("Test Accuracy:",model_xgboost.score(x_test, y_test)*100)
print("Precision Score: ",metrics.precision_score(y_test, y_pred5, pos_label='positive', average='micro'))
print("Recall Score: ",metrics.recall_score(y_test, y_pred5, pos_label='positive', average='micro'))
print("F1 Score: ",metrics.f1_score(y_test, y_pred5, pos_label='positive', average='micro'))
print("Confusion Matrix: ")
print(confusion_matrix(y_test, y_pred5,))

XGboost
Train Accuracy: 0.999537037037037
Test Accuracy: 82.5925925925926
Precision Score:  0.825925925925926
Recall Score:  0.825925925925926
F1 Score:  0.825925925925926
Confusion Matrix: 
[[446  95]
 [ 93 446]]


/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
/home/admin-/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1298: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn("Note that pos_label (set to %r) is ignored when "
